# Make an SED!

### Requirements
- astropy-3.0.2
- bokeh-0.12.6

In [1]:
# Imports
from SEDkit import sed, spectrum, synphot
import astropy.units as q
import numpy as np
from pkg_resources import resource_filename
from bokeh.io import output_notebook, show
output_notebook()

/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:119: UserWarning: Extinction files should be moved to $PYSYN_CDBS/extinction for compatibility with future versions of pysynphot.
  warnings.warn('Extinction files should be moved to '
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/locations.py:155: UserWarning: Extinction files not found in /grp/hst/cdbs/grid/extinction
  warnings.warn('Extinction files not found in %s' % (extdir,))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:114: UserWarning: No graph or component tables found; functionality will be SEVERELY crippled. No files found for /grp/hst/cdbs/mtab/*_tmg.fits.
  'functionality will be SEVERELY crippled. ' + str(e))
/Users/jfilippazzo/miniconda3/envs/astroconda/lib/python3.5/site-packages/pysynphot/refs.py:121: UserWarning: No thermal tables found, no thermal calculations can be performed. No files 

Loading BokehJS ...

Let's create a new SED for an M-dwarf. The software assumes a field age if no `age` argument is given.

In [2]:
s = sed.SED(parallax=(1.7474*q.mas, 0.0501*q.mas), spectral_type='M0V', name='2MASS 261832886')

Make this handle asymmetric uncertainties!
Setting age to (<Quantity 6.0 Gyr>, <Quantity 4.0 Gyr>)


Now let's add some photometry and take a look at how it's stored!

In [3]:
# Add the band name, magnitude, and uncertainty
s.add_photometry('2MASS.J', 14.391, 0.029)
s.add_photometry('2MASS.H', 13.677, 0.032)
s.add_photometry('2MASS.Ks', 13.535, 0.034)
s.add_photometry('WISE.W1', 13.55, 0.027)
s.add_photometry('WISE.W2', 13.581, 0.035)
s.add_photometry('WISE.W3', 12.884, None)
s.add_photometry('SDSS.u', 20.486, 0.073)
s.add_photometry('SDSS.g', 17.779, 0.006)
s.add_photometry('SDSS.r', 16.431, 0.004)
s.add_photometry('SDSS.i', 15.843, 0.004)
s.add_photometry('SDSS.z', 15.514, 0.005)

# Take a look
print(s.photometry)

  band           eff         app_magnitude ...      abs_flux_unc      bandpass
                  um                       ... erg / (Angstrom cm2 s)         
-------- ------------------- ------------- ... ---------------------- --------
  SDSS.u 0.35565239687948447        20.486 ...                    0.0   SDSS.u
  SDSS.g  0.4702495279331733        17.779 ...                    0.0   SDSS.g
  SDSS.r  0.6175578881059804        16.431 ...                    0.0   SDSS.r
  SDSS.i  0.7489976846298569        15.843 ...                    0.0   SDSS.i
  SDSS.z   0.894670956505578        15.514 ...                    0.0   SDSS.z
 2MASS.J  1.2393093155660664        14.391 ...                    0.0  2MASS.J
 2MASS.H  1.6494947091246095        13.677 ...                    0.0  2MASS.H
2MASS.Ks   2.163860596453316        13.535 ...                    0.0 2MASS.Ks
 WISE.W1  3.3897048577485647         13.55 ...                    0.0  WISE.W1
 WISE.W2     4.6406356615375        13.581 ...      

Spectra isn't necessary to build an SED but let's add some for fun.

In [4]:
# Get the spectral data
file = resource_filename('SEDkit', 'data/M0V_spectrum.txt')
wave, flux, unc = np.genfromtxt(file, unpack=True)

# Add the spectrum
s.add_spectrum(wave*q.um, flux*q.W/q.m**2/q.um, unc*q.W/q.m**2/q.um)

# Take a look
show(s.spectra[0].plot)

Warning, 863 of 7982 bins contained negative fluxes; they have been set to zero.


Now that our SED has all the data it needs, we can call the `results` method, which constructs the SED and caluclates the fundamental parameters.

If no `radius` is given, one will be estimated from model isochrones using the calculated `Lbol` and assumed `age`.

In [5]:
print(s.results)

Setting radius to (<Quantity 0.8786209573091851 solRad>, <Quantity 0.06782337214316517 solRad>)
    param           value                unc             units    
------------- ------------------ ------------------- -------------
         name    2MASS 261832886                  --            --
          age                6.0                 4.0           Gyr
     distance             572.28               16.41            pc
     parallax             1.7474              0.0501           mas
       radius 0.8786209573091851 0.06782337214316517        solRad
spectral_type               60.0                 0.5            --
   membership                 --                  --            --
         fbol           1.94e-12            4.99e-15 erg / (cm2 s)
         mbol             17.797               0.003            --
         Lbol           7.61e+31            4.37e+30       erg / s
     Lbol_sun               -1.7               0.025            --
         Mbol              9.009 

In [6]:
fig = s.plot()

But that radius seems wrong. Let's change it...

In [7]:
s.radius = 0.6*q.Rsun, 0.05*q.Rsun

Setting radius to (<Quantity 0.6 solRad>, <Quantity 0.05 solRad>)


In [8]:
print(s.results)

    param           value          unc        units    
------------- ------------------ -------- -------------
         name    2MASS 261832886       --            --
          age                6.0      4.0           Gyr
     distance             572.28    16.41            pc
     parallax             1.7474   0.0501           mas
       radius                0.6     0.05        solRad
spectral_type               60.0      0.5            --
   membership                 --       --            --
         fbol           1.94e-12 4.99e-15 erg / (cm2 s)
         mbol             17.797    0.003            --
         Lbol           7.61e+31 4.37e+30       erg / s
     Lbol_sun               -1.7    0.025            --
         Mbol              9.009    0.031            --
         logg                4.5     0.07            --
         mass 0.8896913720017506      0.0       solMass
         Teff               2798      123             K
